# Digits Recognizer

inspired by the following kernels:
* https://www.kaggle.com/poonaml/deep-neural-network-keras-way
* https://www.kaggle.com/yassineghouzam/introduction-to-cnn-keras-0-997-top-6

## 1. Import necessary Python packages

In [ ]:
import numpy as np
import pandas as pd

from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
%matplotlib inline

## 2. Open datasets

In [ ]:
df_train = pd.read_csv("../input/train.csv")
df_test = pd.read_csv("../input/test.csv")

## 2.1. Check the data shape

In [ ]:
df_train.describe()

In [ ]:
df_test.describe()

## 3. Manipulate data

In [ ]:
y_train = df_train["label"]

In [ ]:
y_train.describe()

In [ ]:
X_train = df_train.drop(columns="label")

In [ ]:
X_train.describe()

In [ ]:
X_test = df_test

## 3.1. Convert pixels to float

In [ ]:
X_train = np.array((df_train.iloc[:,1:].values).astype('float32'))

In [ ]:
X_train.min()

In [ ]:
X_train.max()

In [ ]:
X_test = np.array(df_test.values.astype('float32'))

In [ ]:
X_train.min()

In [ ]:
X_train.max()

## 3.2. Scale Down/Normalize data between 0.00 to 1.00

In [ ]:
X_train = X_train/255.0

In [ ]:
X_train.max()

In [ ]:
X_test = X_test/255.0

In [ ]:
X_test.max()

## 3.3. Convert label digits to integer

In [ ]:
y_train = np.array(df_train.iloc[:,0].values.astype('int32'))

In [ ]:
y_train.min()

In [ ]:
y_train.max()

## 3.4. Reshape Data

In [ ]:
X_train = X_train.reshape(-1, 28, 28, 1)

In [ ]:
X_train.shape

In [ ]:
X_test = X_test.reshape(-1, 28, 28, 1)

In [ ]:
X_test.shape

## 3.4.1. Plot a couple of data for sanity check

In [ ]:
plt.subplot(171)
plt.title(y_train[1])
plt.imshow(X_train[1][:,:,0])

plt.subplot(172)
plt.title(y_train[2])
plt.imshow(X_train[2][:,:,0])

plt.subplot(173)
plt.title(y_train[3])
plt.imshow(X_train[3][:,:,0])

plt.subplot(174)
plt.title(y_train[4])
plt.imshow(X_train[4][:,:,0])

plt.subplot(175)
plt.title(y_train[5])
plt.imshow(X_train[5][:,:,0])

plt.subplot(176)
plt.title(y_train[6])
plt.imshow(X_train[6][:,:,0])

plt.subplot(177)
plt.title(y_train[7])
plt.imshow(X_train[7][:,:,0])

In [ ]:
plt.subplot(151)
plt.imshow(X_test[1][:,:,0])

plt.subplot(152)
plt.imshow(X_test[2][:,:,0])

plt.subplot(153)
plt.imshow(X_test[3][:,:,0])

plt.subplot(154)
plt.imshow(X_test[4][:,:,0])

plt.subplot(155)
plt.imshow(X_test[5][:,:,0])

## 3.5. One-Hot encoding

In [ ]:
from keras.utils.np_utils import to_categorical

y_train_encoded = to_categorical(y_train, num_classes=10)

## 3.5.1. Perform check at some data

In [ ]:
for idx in range (20, 30):
    print (y_train[idx], y_train_encoded[idx])

Good to go!

## 3.6. Split data for Cross Validation

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(X_train, y_train_encoded, test_size = 0.1, random_state=2)

## 4. Design the Classifier

## 4.1. Define the Convolution Neural Network (CNN) model

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense , Dropout , Lambda, Flatten

model = Sequential()

model.add(Conv2D(filters = 32, 
                 kernel_size = (5,5),
                 padding = 'Same', 
                 activation ='relu', 
                 input_shape = (28,28,1)))

model.add(BatchNormalization(axis=1))

model.add(Conv2D(filters = 32, 
                 kernel_size = (5,5),
                 padding = 'Same', 
                 activation ='relu'))

model.add(MaxPool2D(pool_size=(2,2)))
#model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, 
                 kernel_size = (3,3),
                 padding = 'Same', 
                 activation ='relu'))

model.add(BatchNormalization(axis=1))

model.add(Conv2D(filters = 64, 
                 kernel_size = (3,3),
                 padding = 'Same', 
                 activation ='relu'))

model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
#model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(256, activation = "relu"))

model.add(Dropout(0.5))

model.add(Dense(10, activation = "softmax"))

## 4.2. Set Optimizer

In [ ]:
from keras.optimizers import Adam ,RMSprop

optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [ ]:
from keras.callbacks import ReduceLROnPlateau

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

## 4.3. Compile the model

In [ ]:
model.compile(optimizer = optimizer , 
              loss = "categorical_crossentropy", 
              metrics=["accuracy"])

## 4.4. Data Augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
            zoom_range = 0.1, # Randomly zoom image 
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=False,  # randomly flip images
            vertical_flip=False)  # randomly flip images

datagen.fit(X_train)

In [ ]:
batches = datagen.flow(X_train, Y_train, batch_size=96)
val_batches = datagen.flow(X_val, Y_val, batch_size=96)

## 4.5. Fit the model

In [ ]:
history = model.fit_generator(generator=batches, 
                            epochs=1,
                            validation_data=val_batches,
                            validation_steps=val_batches.n,
                            verbose=2,
                            steps_per_epoch=batches.n,
                            callbacks=[learning_rate_reduction])

## 5. Perform Prediction

In [ ]:
predictions = model.predict_classes(X_test, verbose=0)

## 6. Submission

In [ ]:
submissions=pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)),
                          "Label": predictions})

submissions.to_csv("submission.csv", index=False, header=True)